In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib as plt
import os
import pickle
import time
from keras import backend as K
from sklearn.model_selection import cross_val_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.metrics import r2_score
import sklearn.metrics as m
from collections import Counter

Using TensorFlow backend.


In [2]:
'''
Function that takes in a dataframe and then returns the dictionary of accuracies.
'''

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_pred - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def nn_model(localdf,localdict,times):
    for counter in range(times):
        for i in range(len(localdf.columns)):
            df = pd.DataFrame()
            if(i != 0):
                print(localdf.columns[i])
                df = localdf[localdf.columns[i]]
                ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
                ndf.columns = ('t-2', 't-1', 't')
                ndf = ndf.iloc[2:]

                x = ndf[ndf.columns[0:2]]
                y = ndf[ndf.columns[-1:]]

                X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
                model = Sequential()
                model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
                #Output Layer.
                model.add(Dense(1,kernel_initializer='normal'))
                #Compile model.
                model.compile(loss='mean_squared_error',optimizer='adam',metrics=[coeff_determination])
                #Fit the model.
                model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=1)
                #Final Evaluation of the model.
                #model.evaluate returns the 'loss' and the 'metrics' for the model.
                scores = model.evaluate(X_test,y_test,verbose=1)
                #y_pred = model.predict(X_test)
                #print (str(coeff_determination(y_test,y_pred)))
                print ("Metric : %.5f" % (scores[1]))
                localdict.setdefault(counter,[]).append((scores[1]))
                model.summary()
    return localdict            

In [3]:
df=pickle.load(open("load_data_1632.pkl",'rb'),encoding='latin1')
df_new=df

In [4]:
df_new.head()

,dataid,localhour,use,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,...,refrigerator1,refrigerator2,security1,shed1,sprinkler1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1
0,1632,2013-03-15 01:00:00-05,0.820433,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,1632,2013-03-15 02:00:00-05,0.373833,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,1632,2013-03-15 03:00:00-05,0.379117,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,1632,2013-03-15 04:00:00-05,0.341650,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,1632,2013-03-15 05:00:00-05,0.379400,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [5]:
pos=[]
for i in range(3,len(df.columns)):
    if (np.sum(np.isnan(df.iloc[:,i].values))>=40995):
        print ('Empty Column')
        pos.append(i)
    else:
        print ('All fine')
        

All fine
All fine
All fine
Empty Column
Empty Column
All fine
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
All fine
Empty Column
Empty Column
Empty Column
All fine
All fine
All fine
All fine
Empty Column
All fine
All fine
All fine
All fine
Empty Column
All fine
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
All fine
All fine
All fine
All fine
All fine
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
All fine
Empty Column
All fine
All fine
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
All fine
All fine
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
Empty Column
All fine
Empty Column
Empty Column


In [6]:
df_new.drop(df_new.columns[pos], inplace=True, axis=1)

In [7]:
df.columns

Index(['dataid', 'localhour', 'use', 'air1', 'air2', 'air3', 'bathroom1',
       'clotheswasher1', 'dishwasher1', 'disposal1', 'drye1', 'dryg1',
       'furnace1', 'furnace2', 'garage1', 'garage2', 'grid', 'kitchenapp1',
       'kitchenapp2', 'lights_plugs1', 'lights_plugs2', 'lights_plugs3',
       'microwave1', 'outsidelights_plugs1', 'outsidelights_plugs2', 'range1',
       'refrigerator1', 'waterheater1'],
      dtype='object')

In [8]:
for i in range(3,len(df.columns)):
    print (df.columns[i], np.sum(np.isnan(df.iloc[:,i].values)))

air1 1
air2 1
air3 18
bathroom1 18
clotheswasher1 0
dishwasher1 0
disposal1 0
drye1 40833
dryg1 162
furnace1 18
furnace2 0
garage1 18
garage2 162
grid 1
kitchenapp1 0
kitchenapp2 0
lights_plugs1 18
lights_plugs2 18
lights_plugs3 40851
microwave1 0
outsidelights_plugs1 1
outsidelights_plugs2 1
range1 40851
refrigerator1 7942
waterheater1 18


In [9]:
df.shape

(40995, 28)

In [10]:
nuls={}
l=0
for i in range(3,len(df.columns)):
    a=df.iloc[:,i].values
    b=[]
    for j in range (len(a)):
        if np.isnan(a[j])==1:
            b.append(j)
    string = str(i)+"-"+str(np.sum(np.isnan(a)))
    nuls[string]=b


In [11]:
df.head()

,dataid,localhour,use,air1,air2,air3,bathroom1,clotheswasher1,dishwasher1,disposal1,...,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,microwave1,outsidelights_plugs1,outsidelights_plugs2,range1,refrigerator1,waterheater1
0,1632,2013-03-15 01:00:00-05,0.820433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7365,NaN,0.0
1,1632,2013-03-15 02:00:00-05,0.373833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,NaN,0.0
2,1632,2013-03-15 03:00:00-05,0.379117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,NaN,0.0
3,1632,2013-03-15 04:00:00-05,0.341650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,NaN,0.0
4,1632,2013-03-15 05:00:00-05,0.379400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,NaN,0.0


In [12]:
nuls

{'10-40833': [162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226,
  227,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239,
  240,
  241,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  249,
  250,
  251,
  252,
  253,
  254,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  264,
  265,
  266,
  267,
  268,
  269,
  270,
  271,
  272,
  273,
  274,
  275,
  276,
  277,
  278,
  279,
  280,
  281,
  282,
  283,
  284,
  285,
  286,
  287,
  288,
  289,
  290,
  291,
  292,
  293,
  294,
  295,
  296,
  297,
  298,
  299,
  300,
  301,
  302,
 

In [13]:
#After removing the entirely NULL columns.
df.isnull().sum()

dataid                      0
localhour                   0
use                         1
air1                        1
air2                        1
air3                       18
bathroom1                  18
clotheswasher1              0
dishwasher1                 0
disposal1                   0
drye1                   40833
dryg1                     162
furnace1                   18
furnace2                    0
garage1                    18
garage2                   162
grid                        1
kitchenapp1                 0
kitchenapp2                 0
lights_plugs1              18
lights_plugs2              18
lights_plugs3           40851
microwave1                  0
outsidelights_plugs1        1
outsidelights_plugs2        1
range1                  40851
refrigerator1            7942
waterheater1               18
dtype: int64

In [14]:
#Dropping the columns : (1) drye1 (2) lights_plugs3 (3)range1 
df.drop(df.columns[[10,21,25]],inplace=True,axis=1)

In [15]:
#After removing the entirely NULL columns.
df.isnull().sum()

dataid                     0
localhour                  0
use                        1
air1                       1
air2                       1
air3                      18
bathroom1                 18
clotheswasher1             0
dishwasher1                0
disposal1                  0
dryg1                    162
furnace1                  18
furnace2                   0
garage1                   18
garage2                  162
grid                       1
kitchenapp1                0
kitchenapp2                0
lights_plugs1             18
lights_plugs2             18
microwave1                 0
outsidelights_plugs1       1
outsidelights_plugs2       1
refrigerator1           7942
waterheater1              18
dtype: int64

In [16]:
df_thermal = df[['dataid','localhour','air1','air2','air3','furnace1','furnace2','waterheater1']]
#Removing the dataid and the localhour column from df_thermal.
del df_thermal[df_thermal.columns[0]]
print (df_thermal.shape)

(40995, 7)


In [17]:
df_thermal.isnull().sum()

localhour        0
air1             1
air2             1
air3            18
furnace1        18
furnace2         0
waterheater1    18
dtype: int64

In [18]:
df_thermal.head()

,localhour,air1,air2,air3,furnace1,furnace2,waterheater1
0,2013-03-15 01:00:00-05,0.0,0.0,0.0,0.049567,0.0,0.0
1,2013-03-15 02:00:00-05,0.0,0.0,0.0,0.048283,0.0,0.0
2,2013-03-15 03:00:00-05,0.0,0.0,0.0,0.089833,0.0,0.0
3,2013-03-15 04:00:00-05,0.0,0.0,0.0,0.047067,0.0,0.0
4,2013-03-15 05:00:00-05,0.0,0.0,0.0,0.046250,0.0,0.0


In [19]:
df_thermal = pd.DataFrame(df_thermal.values[175:],columns=df_thermal.columns)

In [20]:
df_thermal.isnull().sum()

localhour       0
air1            1
air2            1
air3            0
furnace1        0
furnace2        0
waterheater1    0
dtype: int64

In [21]:
df_thermal.iloc[34491:34494]

,localhour,air1,air2,air3,furnace1,furnace2,waterheater1
34491,2017-02-27 10:00:00-06,0,6.66667e-05,0,0.005,0.005,0
34492,2017-02-27 11:00:00-06,NaN,NaN,0,0.005,0.005,0
34493,2017-02-27 12:00:00-06,0,0,0,0.005,0.005,0


In [22]:
df_thermal.values[34492][1] = (df_thermal.iloc[34492-24*7+1][1] + df_thermal.iloc[34492+24*7+1][1]) / 2
df_thermal.values[34492][2] = (df_thermal.iloc[34292-24*7+1][2] + df_thermal.iloc[34492+24*7+1][2]) / 2

In [23]:
df_thermal.iloc[34316-24*7+1]

localhour       2017-02-13 04:00:00-06
air1                                 0
air2                                 0
air3                                 0
furnace1                         0.005
furnace2                         0.005
waterheater1                         0
Name: 34149, dtype: object

In [24]:
df_thermal.iloc[34316+24*7+1]

localhour       2017-02-27 04:00:00-06
air1                                 0
air2                                 0
air3                                 0
furnace1                         0.005
furnace2                         0.005
waterheater1                         0
Name: 34485, dtype: object

In [30]:
df_thermal.head()

,localhour,air1,air2,air3,furnace1,furnace2,waterheater1
0,2013-03-23 08:00:00-05,0,0,0,0.0059,0.01,0
1,2013-03-23 09:00:00-05,0,0,0,0.00491667,0.01,0
2,2013-03-23 10:00:00-05,0,0,0,0.0059,0.0762833,0.181567
3,2013-03-23 11:00:00-05,0,0,0,0.0059,0.0760333,1.32477
4,2013-03-23 12:00:00-05,0,0,0,0.006,0.01,0


In [31]:
thermal_dict = {}
ti = time.time()
thermal_dict = nn_model(df_thermal,thermal_dict,1)
tf = time.time()
print ("Time Taken : ",tf-ti)
print (thermal_dict)

air1
Epoch 1/200
32654/32654 [==============================] - 1s - loss: 0.1514 - coeff_determination: -24.1511     
Epoch 2/200
32654/32654 [==============================] - 0s - loss: 0.0979 - coeff_determination: -18.6972     
Epoch 3/200
32654/32654 [==============================] - 0s - loss: 0.0581 - coeff_determination: -0.8562     
Epoch 4/200
32654/32654 [==============================] - 0s - loss: 0.0424 - coeff_determination: 0.4304     
Epoch 5/200
32654/32654 [==============================] - 0s - loss: 0.0403 - coeff_determination: 0.5682     
Epoch 6/200
32654/32654 [==============================] - 0s - loss: 0.0397 - coeff_determination: 0.5926     
Epoch 7/200
32654/32654 [==============================] - 0s - loss: 0.0392 - coeff_determination: 0.6015     
Epoch 8/200
32654/32654 [==============================] - 0s - loss: 0.0388 - coeff_determination: 0.6110     
Epoch 9/200
32654/32654 [==============================] - 0s - loss: 0.0384 - coeff_determina

32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6367     
Epoch 144/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6357     
Epoch 145/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6333     
Epoch 146/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6357     
Epoch 147/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6356     
Epoch 148/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6378     
Epoch 149/200
32654/32654 [==============================] - 0s - loss: 0.0371 - coeff_determination: 0.6326     
Epoch 150/200
32654/32654 [==============================] - 0s - loss: 0.0372 - coeff_determination: 0.6370     
Epoch 151/200
32654/32654 [==============================] - 0s - loss: 0.0371 - coeff_determination: 

32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2237     
Epoch 80/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2327     
Epoch 81/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2004     
Epoch 82/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2177     
Epoch 83/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2085     
Epoch 84/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2261     
Epoch 85/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2187     
Epoch 86/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2298     
Epoch 87/200
32654/32654 [==============================] - 0s - loss: 0.0181 - coeff_determination: 0.2247  

32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5764     
Epoch 14/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5710     
Epoch 15/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5721     
Epoch 16/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5650     
Epoch 17/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5777     
Epoch 18/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5772     
Epoch 19/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5646     
Epoch 20/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5882     
Epoch 21/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5739  

32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5871     
Epoch 157/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5758     
Epoch 158/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5930     
Epoch 159/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5956     
Epoch 160/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5776     
Epoch 161/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5749     
Epoch 162/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5733     
Epoch 163/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 0.5786     
Epoch 164/200
32654/32654 [==============================] - 0s - loss: 0.0114 - coeff_determination: 

32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5899     
Epoch 22/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5801     
Epoch 23/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5828     
Epoch 24/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5817     
Epoch 25/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5833     
Epoch 26/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.6133     
Epoch 27/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5994     
Epoch 28/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.6121     
Epoch 29/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5789  

32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.6054     
Epoch 164/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5902     
Epoch 165/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5854     
Epoch 166/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5847     
Epoch 167/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.6019     
Epoch 168/200
32654/32654 [==============================] - 0s - loss: 0.0511 - coeff_determination: 0.5729     - ETA: 0s - loss: 0.0467 - coeff_determinati
Epoch 169/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5981     
Epoch 170/200
32654/32654 [==============================] - 0s - loss: 0.0510 - coeff_determination: 0.5980     
Epoch 171/200
32654/32654 [==============================]

32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5551     
Epoch 100/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5411     
Epoch 101/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5461     - ETA: 0s - loss: 0.0012 - coeff_determinatio
Epoch 102/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5653     
Epoch 103/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5537     
Epoch 104/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5787     
Epoch 105/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5659     
Epoch 106/200
32654/32654 [==============================] - 0s - loss: 0.0013 - coeff_determination: 0.5396     
Epoch 107/200
32654/32654 [==============================

32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -46.5000     
Epoch 33/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -52.4114     
Epoch 34/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -50.9850     
Epoch 35/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -50.8538     
Epoch 36/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -49.5379     
Epoch 37/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -53.9574     
Epoch 38/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -51.2475     
Epoch 39/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determination: -51.1297     
Epoch 40/200
32654/32654 [==============================] - 0s - loss: 0.0344 - coeff_determi

32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -31.1775     
Epoch 174/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -28.6800     
Epoch 175/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -28.2065     
Epoch 176/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -29.0183     
Epoch 177/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -29.4020     
Epoch 178/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -28.8244     
Epoch 179/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -28.2213     
Epoch 180/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff_determination: -27.5808     
Epoch 181/200
32654/32654 [==============================] - 0s - loss: 0.0334 - coeff

In [44]:
#With the 'metric' as 'coefficient of determination (r^2)'.
'''
water-heater attribute is always negative when r^2 is used.
'''
thermal_dict1 = thermal_dict
print (thermal_dict1)


{0: [0.57281007940777373, -1.0648682339117843, 0.4171872332889271, 0.33941760762890782, 0.2714133022580294, -32.898724590195911]}


In [33]:
#With the 'metric' as 'mean_squared_error'.
thermal_dict3 = thermal_dict
print (thermal_dict3)

{0: [0.57281007940777373, -1.0648682339117843, 0.4171872332889271, 0.33941760762890782, 0.2714133022580294, -32.898724590195911]}


In [34]:
'''
Moving on to 'Extra Trees Regresssor.
'''
def et_month(localdf):
    scores=[]
    for i in range(len(localdf.columns)):
                df = pd.DataFrame()
                if(i != 0):
                    #print(localdf.columns[i])
                    df = localdf[localdf.columns[i]]
                    ndf = pd.concat([df.shift(168), df.shift(24), df], axis=1)
                    ndf.columns = ('t-168', 't-24', 't')
                    ndf = ndf.iloc[168:]

                    x = ndf[ndf.columns[0:2]]
                    y = ndf[ndf.columns[-1:]]
                    
                    #Sequential Split.
                    train_size = int(70*x.shape[0]/100)
                    X_train = x.values[:train_size]
                    y_train = y.values[:train_size]
                    
                    X_test = x.values[train_size:]
                    y_test = y.values[train_size:]
                    
                    print (X_train.shape,y_train.shape)
                    print (X_test.shape,y_test.shape)
                    #X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)

                    model = ExtraTreeRegressor()
                    model = model.fit(X_train,y_train)
                    y_pred = model.predict(X_test)
                    #scores.append(r2_score(y_test,y_pred))
                    scores.append(m.mean_squared_error(y_test,y_pred))
                    #scores.append(cross_val_score(model,x.values,y.values,scoring='r2',cv=2))
    return scores          

In [33]:
'''
Moving on to 'Extra Trees Regresssor.
'''
def et_2hours(localdf):
    scores=[]
    for i in range(len(localdf.columns)):
                df = pd.DataFrame()
                if(i != 0):
                    #print(localdf.columns[i])
                    df = localdf[localdf.columns[i]]
                    ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
                    ndf.columns = ('t-2', 't-1', 't')
                    ndf = ndf.iloc[2:]

                    x = ndf[ndf.columns[0:2]]
                    y = ndf[ndf.columns[-1:]]
                    
                    #Sequential Split.
                    train_size = int(70*x.shape[0]/100)
                    X_train = x.values[:train_size]
                    y_train = y.values[:train_size]
                    
                    X_test = x.values[train_size:]
                    y_test = y.values[train_size:]
                    
                    print (X_train.shape,y_train.shape)
                    print (X_test.shape,y_test.shape)
                    #X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)

                    model = ExtraTreeRegressor()
                    model = model.fit(X_train,y_train)
                    y_pred = model.predict(X_test)
                    #scores.append(r2_score(y_test,y_pred))
                    scores.append(m.mean_squared_error(y_test,y_pred))
                    #scores.append(cross_val_score(model,x.values,y.values,scoring='r2',cv=2))
    return scores          

In [37]:
'''
Moving on to 'Extra Trees Regresssor.
'''
def et_4hours(localdf):
    scores=[]
    for i in range(len(localdf.columns)):
                df = pd.DataFrame()
                if(i != 0):
                    #print(localdf.columns[i])
                    df = localdf[localdf.columns[i]]
                    ndf = pd.concat([df.shift(4),df.shift(3),df.shift(2), df.shift(1), df], axis=1)
                    ndf.columns = ('t-4','t-3','t-2', 't-1', 't')
                    ndf = ndf.iloc[4:]

                    x = ndf[ndf.columns[0:4]]
                    y = ndf[ndf.columns[-1:]]
                    
                    #Sequential Split.
                    train_size = int(70*x.shape[0]/100)
                    X_train = x.values[:train_size]
                    y_train = y.values[:train_size]

                    X_test = x.values[train_size:]
                    y_test = y.values[train_size:]
                    
                    print (X_train.shape,y_train.shape)
                    print (X_test.shape,y_test.shape)
                    
                    #X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)

                    model = ExtraTreeRegressor()
                    model = model.fit(X_train,y_train)
                    y_pred = model.predict(X_test)
                    #scores.append(r2_score(y_test,y_pred))
                    scores.append(m.mean_squared_error(y_test,y_pred))
                    #scores.append(cross_val_score(model,x.values,y.values,scoring='r2',cv=2))
    return scores          

In [31]:
#Metric : 'r^2'.
scores_2hours = et_2hours(df_thermal)
print (scores_2hours)

[array([ 0.57303293,  0.58185637]), array([ 0.36696033,  0.37221549]), array([ 0.6319369 ,  0.48454331]), array([ 0.60044037,  0.49502702]), array([ 0.46097493,  0.31680585]), array([-0.36939523, -0.24388865])]


In [32]:
#Metric : 'r^2'.
scores_4hours = et_4hours(df_thermal)
print (scores_4hours)

[array([ 0.56846882,  0.56787801]), array([ 0.39209297,  0.32989165]), array([ 0.65910766,  0.4908278 ]), array([ 0.60796577,  0.44329891]), array([-1.87280973,  0.33262254]), array([-0.4520411 , -0.40125901])]


In [33]:
#Metric : 'r^2'.
scores_month = et_month(df_thermal)
print (scores_month)

[array([ 0.37674988,  0.31931007]), array([ 0.08206034,  0.0377291 ]), array([ 0.27609219,  0.21993242]), array([-0.02126532, -0.23019628]), array([ 0.26208185, -0.18036948]), array([-0.70714878, -0.44804766])]


In [35]:
#Metric : 'mse'.
scores_2hours = et_2hours(df_thermal)
print (scores_2hours)

(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
(28572, 2) (28572, 1)
(12246, 2) (12246, 1)
[0.066291323054328152, 0.031763438050504807, 0.010161939693149501, 0.039744507992379517, 0.0010132881635656367, 0.037236108758228512]


In [38]:
#Metric : 'mse'.
scores_4hours = et_4hours(df_thermal)
print (scores_4hours)

(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
(28571, 4) (28571, 1)
(12245, 4) (12245, 1)
[0.065919164765892491, 0.031218910736631401, 0.0092476137970330663, 0.043824699368812878, 0.0010705180808068219, 0.03954773926072884]


In [40]:
df_thermal.head(10)

,localhour,air1,air2,air3,furnace1,furnace2,waterheater1
0,2013-03-23 08:00:00-05,0,0,0,0.0059,0.01,0
1,2013-03-23 09:00:00-05,0,0,0,0.00491667,0.01,0
2,2013-03-23 10:00:00-05,0,0,0,0.0059,0.0762833,0.181567
3,2013-03-23 11:00:00-05,0,0,0,0.0059,0.0760333,1.32477
4,2013-03-23 12:00:00-05,0,0,0,0.006,0.01,0
5,2013-03-23 13:00:00-05,0,0,0,0.006,0.01,0
6,2013-03-23 14:00:00-05,0,0,0,0.006,0.01,0
7,2013-03-23 15:00:00-05,0,0,0,0.006,0.01,0
8,2013-03-23 16:00:00-05,0,0,0,0.006,0.01,0
9,2013-03-23 17:00:00-05,0,0,0,0.006,0.01,0


In [34]:
df_thermal.to_csv("df_thermal.csv",encoding="utf8")

In [35]:
for i in df_thermal.columns:
    print (np.amax(df_thermal[i]))

2017-11-18 02:00:00-06
2.399916666666667
1.9250166666666668
1.5737333333333332
5.048083333333333
1.0207
1.9665833333333331


In [46]:
df_thermal.shape

(40820, 7)

In [45]:
df_thermal.tail(10)

,localhour,air1,air2,air3,furnace1,furnace2,waterheater1
40810,2017-11-17 17:00:00-06,0,0,0,0.005,0.005,0
40811,2017-11-17 18:00:00-06,0,0,0,0.005,0.005,0
40812,2017-11-17 19:00:00-06,0,0,0,0.005,0.005,0
40813,2017-11-17 20:00:00-06,0,0,0,0.005,0.005,0
40814,2017-11-17 21:00:00-06,0,0,0.00055,0.00515,0.005,0.415017
40815,2017-11-17 22:00:00-06,0,0,0,0.005,0.005,0
40816,2017-11-17 23:00:00-06,0,0,0,0.005,0.005,0
40817,2017-11-18 00:00:00-06,0,0,0,0.005,0.005,0
40818,2017-11-18 01:00:00-06,0,0,0,0.005,0.005,0
40819,2017-11-18 02:00:00-06,0,0,0,0.005,0.005,0


In [ ]:
days = ['Saturday','Sunday','Monday','Tuesday','Wednesday','Thursday','Friday']
columns = np.empty([df_thermal.shape[0],1],dtype=str)
i=0
while i <= (df_thermal.shape[0]-7):
    columns[i]   = 'Saturday'
    columns[i+1] = 'Sunday'
    columns[i+2] = 'Monday'
    columns[i+3] = 'Tuesday'
    columns[i+4] = 'Wednesday'
    columns[i+5] = 'Thursday'
    columns[i+6] = 'Friday'
    i += 7

In [54]:
print (columns[0])

['S']
